# Step 2: 퍼셉트론 (Perceptron) - 신경망의 기초

퍼셉트론은 가장 간단한 형태의 인공 신경망입니다. 1957년 Frank Rosenblatt이 제안한 이 모델은 현대 딥러닝의 기초가 되었습니다.

## 학습 목표
1. 퍼셉트론의 구조와 동작 원리 이해
2. 선형 분류 문제 해결
3. 경사하강법으로 가중치 학습
4. 퍼셉트론의 한계점 이해

## 1. 퍼셉트론이란?

퍼셉트론은 여러 입력을 받아 하나의 출력을 내는 알고리즘입니다.

### 수식으로 표현하면:
$$y = f(\sum_{i=1}^{n} w_i x_i + b)$$

- $x_i$: 입력값
- $w_i$: 가중치 (weight)
- $b$: 편향 (bias)
- $f$: 활성화 함수 (activation function)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 시각화를 위한 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

## 2. 간단한 퍼셉트론 구현

먼저 AND 게이트를 구현해봅시다.

In [ ]:
# AND 게이트 구현
def AND(x1, x2):
    # 가중치와 편향 (미리 계산된 값)
    w1, w2 = 0.5, 0.5
    b = -0.7
    
    # 가중합 계산
    tmp = x1*w1 + x2*w2 + b
    
    # 활성화 함수 (계단 함수)
    if tmp <= 0:
        return 0
    else:
        return 1

# 테스트
print("AND 게이트 결과:")
print(f"AND(0, 0) = {AND(0, 0)}")
print(f"AND(1, 0) = {AND(1, 0)}")
print(f"AND(0, 1) = {AND(0, 1)}")
print(f"AND(1, 1) = {AND(1, 1)}")

### 2.1 OR 게이트와 NAND 게이트

In [ ]:
# OR 게이트
def OR(x1, x2):
    w1, w2 = 0.5, 0.5
    b = -0.2  # AND와 다른 편향값
    
    tmp = x1*w1 + x2*w2 + b
    return 0 if tmp <= 0 else 1

# NAND 게이트 (NOT AND)
def NAND(x1, x2):
    w1, w2 = -0.5, -0.5  # 음수 가중치
    b = 0.7
    
    tmp = x1*w1 + x2*w2 + b
    return 0 if tmp <= 0 else 1

# 테스트
print("논리 게이트 진리표:")
print("x1 | x2 | AND | OR | NAND")
print("-" * 25)
for x1, x2 in [(0, 0), (0, 1), (1, 0), (1, 1)]:
    print(f" {x1} |  {x2} |  {AND(x1, x2)}  |  {OR(x1, x2)} |  {NAND(x1, x2)}")

## 3. 퍼셉트론의 한계: XOR 문제

단층 퍼셉트론은 선형 분리가 불가능한 문제를 해결할 수 없습니다.

In [ ]:
# XOR 데이터 시각화
X_xor = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_xor = np.array([0, 1, 1, 0])  # XOR 출력

# 시각화
plt.figure(figsize=(10, 4))

# AND 게이트 (선형 분리 가능)
plt.subplot(1, 3, 1)
y_and = np.array([0, 0, 0, 1])
plt.scatter(X_xor[y_and == 0][:, 0], X_xor[y_and == 0][:, 1], c='red', s=100, label='0')
plt.scatter(X_xor[y_and == 1][:, 0], X_xor[y_and == 1][:, 1], c='blue', s=100, label='1')
plt.plot([0, 1.5], [1.5, 0], 'k--', alpha=0.5)  # 결정 경계
plt.title('AND Gate (Linear Separable)')
plt.xlabel('x1')
plt.ylabel('x2')
plt.xlim(-0.5, 1.5)
plt.ylim(-0.5, 1.5)
plt.legend()
plt.grid(True, alpha=0.3)

# OR 게이트 (선형 분리 가능)
plt.subplot(1, 3, 2)
y_or = np.array([0, 1, 1, 1])
plt.scatter(X_xor[y_or == 0][:, 0], X_xor[y_or == 0][:, 1], c='red', s=100, label='0')
plt.scatter(X_xor[y_or == 1][:, 0], X_xor[y_or == 1][:, 1], c='blue', s=100, label='1')
plt.plot([-0.5, 1.5], [1, -1], 'k--', alpha=0.5)  # 결정 경계
plt.title('OR Gate (Linear Separable)')
plt.xlabel('x1')
plt.ylabel('x2')
plt.xlim(-0.5, 1.5)
plt.ylim(-0.5, 1.5)
plt.legend()
plt.grid(True, alpha=0.3)

# XOR 게이트 (선형 분리 불가능)
plt.subplot(1, 3, 3)
plt.scatter(X_xor[y_xor == 0][:, 0], X_xor[y_xor == 0][:, 1], c='red', s=100, label='0')
plt.scatter(X_xor[y_xor == 1][:, 0], X_xor[y_xor == 1][:, 1], c='blue', s=100, label='1')
plt.title('XOR Gate (NOT Linear Separable!)')
plt.xlabel('x1')
plt.ylabel('x2')
plt.xlim(-0.5, 1.5)
plt.ylim(-0.5, 1.5)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. 학습 가능한 퍼셉트론 구현

이제 가중치를 자동으로 학습하는 퍼셉트론을 구현해봅시다.

In [ ]:
class Perceptron:
    def __init__(self, input_size, learning_rate=0.01):
        """
        퍼셉트론 초기화
        
        Parameters:
        - input_size: 입력 특징의 개수
        - learning_rate: 학습률
        """
        # 가중치를 작은 무작위 값으로 초기화
        self.weights = np.random.randn(input_size) * 0.1
        self.bias = 0.0
        self.learning_rate = learning_rate
    
    def predict(self, X):
        """
        입력에 대한 예측값 계산
        """
        # 가중합 계산: w·x + b
        z = np.dot(X, self.weights) + self.bias
        # 활성화 함수 (계단 함수)
        return np.where(z > 0, 1, 0)
    
    def train(self, X, y, epochs=100):
        """
        퍼셉트론 학습 알고리즘
        """
        errors = []
        
        for epoch in range(epochs):
            total_error = 0
            
            # 각 샘플에 대해
            for xi, yi in zip(X, y):
                # 예측
                prediction = self.predict(xi.reshape(1, -1))[0]
                
                # 오차 계산
                error = yi - prediction
                total_error += abs(error)
                
                # 가중치 업데이트 (오차가 있을 때만)
                if error != 0:
                    self.weights += self.learning_rate * error * xi
                    self.bias += self.learning_rate * error
            
            errors.append(total_error)
            
            # 수렴 확인
            if total_error == 0:
                print(f"수렴! (epoch {epoch + 1})")
                break
        
        return errors

### 4.1 AND 게이트 학습

In [ ]:
# AND 게이트 데이터
X_and = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_and = np.array([0, 0, 0, 1])

# 퍼셉트론 생성 및 학습
perceptron_and = Perceptron(input_size=2, learning_rate=0.1)
errors = perceptron_and.train(X_and, y_and, epochs=20)

# 학습 과정 시각화
plt.figure(figsize=(12, 4))

# 오차 변화
plt.subplot(1, 3, 1)
plt.plot(errors, 'b-o')
plt.xlabel('Epoch')
plt.ylabel('Total Error')
plt.title('Learning Curve')
plt.grid(True, alpha=0.3)

# 학습된 결정 경계
plt.subplot(1, 3, 2)
plt.scatter(X_and[y_and == 0][:, 0], X_and[y_and == 0][:, 1], c='red', s=100, label='0')
plt.scatter(X_and[y_and == 1][:, 0], X_and[y_and == 1][:, 1], c='blue', s=100, label='1')

# 결정 경계 그리기
x_range = np.linspace(-0.5, 1.5, 100)
if perceptron_and.weights[1] != 0:
    y_boundary = -(perceptron_and.weights[0] * x_range + perceptron_and.bias) / perceptron_and.weights[1]
    plt.plot(x_range, y_boundary, 'k--', alpha=0.5, label='Decision Boundary')

plt.xlabel('x1')
plt.ylabel('x2')
plt.title('Learned Decision Boundary')
plt.xlim(-0.5, 1.5)
plt.ylim(-0.5, 1.5)
plt.legend()
plt.grid(True, alpha=0.3)

# 예측 결과
plt.subplot(1, 3, 3)
predictions = perceptron_and.predict(X_and)
plt.table(cellText=[[x1, x2, y, pred] for (x1, x2), y, pred in zip(X_and, y_and, predictions)],
          colLabels=['x1', 'x2', 'Target', 'Prediction'],
          cellLoc='center',
          loc='center')
plt.axis('off')
plt.title('Prediction Results')

plt.tight_layout()
plt.show()

print(f"\n학습된 가중치: w1={perceptron_and.weights[0]:.3f}, w2={perceptron_and.weights[1]:.3f}")
print(f"학습된 편향: b={perceptron_and.bias:.3f}")

## 5. 다층 퍼셉트론으로 XOR 문제 해결

XOR 문제는 여러 개의 퍼셉트론을 조합하면 해결할 수 있습니다.

In [ ]:
def XOR(x1, x2):
    """
    다층 퍼셉트론으로 XOR 구현
    XOR = (x1 OR x2) AND (x1 NAND x2)
    """
    s1 = OR(x1, x2)
    s2 = NAND(x1, x2)
    y = AND(s1, s2)
    return y

# 테스트
print("XOR 게이트 (다층 퍼셉트론):")
print(f"XOR(0, 0) = {XOR(0, 0)}")
print(f"XOR(1, 0) = {XOR(1, 0)}")
print(f"XOR(0, 1) = {XOR(0, 1)}")
print(f"XOR(1, 1) = {XOR(1, 1)}")

# 네트워크 구조 시각화
plt.figure(figsize=(10, 6))
ax = plt.gca()

# 노드 위치
input_x = [0, 0]
input_y = [1, 0]
hidden_x = [1, 1]
hidden_y = [1, 0]
output_x = [2]
output_y = [0.5]

# 노드 그리기
plt.scatter(input_x, input_y, s=500, c='lightblue', edgecolors='black', linewidths=2)
plt.scatter(hidden_x, hidden_y, s=500, c='lightgreen', edgecolors='black', linewidths=2)
plt.scatter(output_x, output_y, s=500, c='lightcoral', edgecolors='black', linewidths=2)

# 연결선 그리기
for i in range(2):
    for j in range(2):
        plt.plot([input_x[i], hidden_x[j]], [input_y[i], hidden_y[j]], 'k-', alpha=0.5)
for j in range(2):
    plt.plot([hidden_x[j], output_x[0]], [hidden_y[j], output_y[0]], 'k-', alpha=0.5)

# 라벨
plt.text(0, 1.2, 'x1', fontsize=12, ha='center')
plt.text(0, -0.2, 'x2', fontsize=12, ha='center')
plt.text(1, 1.2, 'OR', fontsize=12, ha='center')
plt.text(1, -0.2, 'NAND', fontsize=12, ha='center')
plt.text(2, 0.7, 'AND', fontsize=12, ha='center')
plt.text(2.5, 0.5, 'XOR Output', fontsize=12, ha='left')

plt.xlim(-0.5, 3)
plt.ylim(-0.5, 1.5)
plt.axis('off')
plt.title('Multi-layer Perceptron for XOR', fontsize=14)
plt.show()

## 6. 실제 데이터로 퍼셉트론 학습하기

2차원 데이터를 생성하여 이진 분류 문제를 해결해봅시다.

In [ ]:
# 선형 분리 가능한 데이터 생성
np.random.seed(42)
n_samples = 100

# 클래스 0: 좌하단
X_class0 = np.random.randn(n_samples // 2, 2) - [2, 2]
y_class0 = np.zeros(n_samples // 2)

# 클래스 1: 우상단
X_class1 = np.random.randn(n_samples // 2, 2) + [2, 2]
y_class1 = np.ones(n_samples // 2)

# 데이터 합치기
X = np.vstack([X_class0, X_class1])
y = np.hstack([y_class0, y_class1]).astype(int)

# 데이터 섞기
indices = np.random.permutation(len(X))
X, y = X[indices], y[indices]

# 퍼셉트론 학습
perceptron = Perceptron(input_size=2, learning_rate=0.01)
errors = perceptron.train(X, y, epochs=50)

# 결과 시각화
plt.figure(figsize=(12, 5))

# 학습 곡선
plt.subplot(1, 2, 1)
plt.plot(errors, 'b-')
plt.xlabel('Epoch')
plt.ylabel('Total Error')
plt.title('Learning Curve')
plt.grid(True, alpha=0.3)

# 분류 결과
plt.subplot(1, 2, 2)
plt.scatter(X[y == 0][:, 0], X[y == 0][:, 1], c='red', alpha=0.6, label='Class 0')
plt.scatter(X[y == 1][:, 0], X[y == 1][:, 1], c='blue', alpha=0.6, label='Class 1')

# 결정 경계
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                     np.linspace(y_min, y_max, 100))
Z = perceptron.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.contourf(xx, yy, Z, alpha=0.3, cmap='RdBu')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Perceptron Classification Result')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 정확도 계산
predictions = perceptron.predict(X)
accuracy = np.mean(predictions == y)
print(f"\n학습 정확도: {accuracy * 100:.2f}%")

## 7. 퍼셉트론의 활성화 함수들

지금까지는 계단 함수를 사용했지만, 다른 활성화 함수들도 살펴봅시다.

In [ ]:
# 다양한 활성화 함수
x = np.linspace(-5, 5, 100)

# 계단 함수
def step_function(x):
    return np.where(x > 0, 1, 0)

# 시그모이드 함수
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# tanh 함수
def tanh(x):
    return np.tanh(x)

# ReLU 함수
def relu(x):
    return np.maximum(0, x)

# 시각화
plt.figure(figsize=(12, 8))

functions = [
    (step_function, 'Step Function', 'red'),
    (sigmoid, 'Sigmoid', 'blue'),
    (tanh, 'Tanh', 'green'),
    (relu, 'ReLU', 'orange')
]

for i, (func, name, color) in enumerate(functions, 1):
    plt.subplot(2, 2, i)
    y = func(x)
    plt.plot(x, y, color=color, linewidth=2)
    plt.grid(True, alpha=0.3)
    plt.xlabel('x')
    plt.ylabel(f'{name}(x)')
    plt.title(name)
    plt.axhline(y=0, color='k', linewidth=0.5)
    plt.axvline(x=0, color='k', linewidth=0.5)

plt.tight_layout()
plt.show()

## 8. 연습 문제

In [ ]:
# 문제 1: NOT 게이트 구현
# 힌트: 입력이 하나이고, 입력이 0이면 1을, 1이면 0을 출력
def NOT(x):
    # 여기에 코드 작성
    pass

# 문제 2: 시그모이드 활성화 함수를 사용하는 퍼셉트론 구현
class SigmoidPerceptron:
    def __init__(self, input_size, learning_rate=0.01):
        self.weights = np.random.randn(input_size) * 0.1
        self.bias = 0.0
        self.learning_rate = learning_rate
    
    def sigmoid(self, x):
        # 시그모이드 함수 구현
        pass
    
    def predict(self, X):
        # 시그모이드를 사용한 예측
        pass
    
    def train(self, X, y, epochs=100):
        # 경사하강법으로 학습
        pass

# 문제 3: 3개의 클래스를 분류하는 다중 퍼셉트론 구현
# 힌트: One-vs-All 전략 사용

## 정리

이번 튜토리얼에서 배운 내용:
1. 퍼셉트론의 기본 구조와 동작 원리
2. 논리 게이트 구현 (AND, OR, NAND)
3. 퍼셉트론의 한계 (XOR 문제)
4. 학습 알고리즘 구현
5. 다층 퍼셉트론으로 비선형 문제 해결
6. 다양한 활성화 함수

### 핵심 개념:
- **선형 분리성**: 단층 퍼셉트론은 선형으로 분리 가능한 문제만 해결 가능
- **가중치 학습**: 오차를 이용한 가중치 업데이트
- **다층 구조**: 여러 퍼셉트론을 조합하여 복잡한 문제 해결

다음 단계에서는 이를 확장하여 다층 퍼셉트론(MLP)과 역전파 알고리즘을 구현해보겠습니다!